In [1]:
!pip install implicit

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\serge\anaconda3\python.exe' 'C:\Users\serge\anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\serge\AppData\Local\Temp\tmpc44vd3r2'
       cwd: C:\Users\serge\AppData\Local\Temp\pip-install-wp18q_h5\implicit
  Complete output (297 lines):
  Not searching for unused variables given on the command line.
  CMake Error: CMake was unable to find a build program corresponding to "Ninja".  CMAKE_MAKE_PROGRAM is not set.  You probably need to select a different build tool.
  -- Configuring incomplete, errors occurred!
  See also "C:/Users/serge/AppData/Local/Temp/pip-install-wp18q_h5/implicit/_cmake_test_compile/build/CMakeFiles/CMakeOutput.log".
  Not searching for unused variables given on the command line.
  CMake Error at CMakeLists.txt:2 (PROJECT):
    Generator
  
      Visual Studio 17 2022
  
    could not find any instance of Visual Studio.
  
  
  
  -- Configuring incomplete, er


  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build implicit


  -- Trying "Ninja (Visual Studio 17 2022 x64 v143)" generator
  --------------------------------
  ---------------------------
  ----------------------
  -----------------
  ------------
  -------
  --
  --
  -------
  ------------
  -----------------
  ----------------------
  ---------------------------
  --------------------------------
  -- Trying "Ninja (Visual Studio 17 2022 x64 v143)" generator - failure
  --------------------------------------------------------------------------------
  
  
  
  --------------------------------------------------------------------------------
  -- Trying "Visual Studio 17 2022 x64 v143" generator
  --------------------------------
  ---------------------------
  ----------------------
  -----------------
  ------------
  -------
  --
  --
  -------
  ------------
  -----------------
  ----------------------
  ---------------------------
  --------------------------------
  -- Trying "Visual Studio 17 2022 x64 v143" generator - failure
  -------

## Задание 1

In [2]:
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


In [7]:
items_weights= data_train[['item_id','sales_value']]
items_weights.head(5)

,item_id,sales_value
0,1004906,1.39
1,1033142,0.82
2,1036325,0.99
3,1082185,1.21
4,8160430,1.50


In [8]:
items_weights.loc[items_weights['sales_value'] == 0, 'sales_value'] = 0.001

items_weights['weight'] = np.log(items_weights['sales_value'])
items_weights.loc[(items_weights['weight'] < 0)] = 0

min_value = items_weights['weight'].min()
max_value = items_weights['weight'].max()
sum_1 = items_weights['weight'].sum()
items_weights = items_weights[['item_id','weight']].groupby('item_id').sum()

items_weights['weight'] = items_weights['weight']/sum_1
items_weights['item_id'] = items_weights.index

items_weights.head(5)

,weight,item_id
item_id,,
0,0.000000e+00,0
25671,2.704388e-06,25671
26093,2.441816e-07,26093
26190,2.273573e-07,26190
26355,3.596883e-07,26355


In [9]:
def weighted_random_recommendation(items_weights, n=5):
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights)
    
    return recs

In [10]:
%%time
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(5)

Wall time: 4.8 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12810466, 1147273, 824796, 7168833, 7025203]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[823862, 7157113, 1075074, 941741, 2424234]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6533765, 1018818, 7440992, 1097661, 847962]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9487306, 1048445, 7406474, 985889, 847982]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[959987, 9803068, 919681, 5572234, 5567143]"


## Задание 2

In [11]:
popular_items = data_train.groupby('item_id')['quantity', 'sales_value'].sum().reset_index()
popular_items.rename(columns={'quantity': 'n_sold'}, inplace=True)
popular_items.head(5)

,item_id,n_sold,sales_value
0,25671,6,20.94
1,26081,1,0.99
2,26093,1,1.59
3,26190,1,1.54
4,26355,2,1.98


In [12]:
top_5000 = popular_items.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
top_5000[:2]

[6534178, 6533889]

In [13]:
weighted_top_5000 = popular_items.sort_values('n_sold', ascending=False).head(5000)
weighted_top_5000.head(5)

,item_id,n_sold,sales_value
55470,6534178,190227964,447799.94
55430,6533889,15978434,40483.34
55465,6534166,12439291,30170.77
55576,6544236,2501949,6584.34
43620,1404121,1562004,3705.77


In [14]:
def random_recommendation(items, n=5):
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [15]:
def popularity_recommendation(data, n=5):
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [16]:
%%time
popular_recs = popularity_recommendation(data_train, n=5)
result['random_recommendation_top_5000'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))
result['popular_recommendation_top_5000'] = result['user_id'].apply(lambda x: popular_recs)
result.head(5)

Wall time: 1.81 s


,user_id,actual,weighted_random_recommendation,random_recommendation_top_5000,popular_recommendation_top_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12810466, 1147273, 824796, 7168833, 7025203]","[1080472, 933067, 9445919, 866950, 1007796]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[823862, 7157113, 1075074, 941741, 2424234]","[9835509, 12352204, 947200, 965530, 986156]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6533765, 1018818, 7440992, 1097661, 847962]","[867667, 863885, 1012043, 829617, 1072330]","[6534178, 6533889, 1029743, 6534166, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9487306, 1048445, 7406474, 985889, 847982]","[12349795, 6464056, 1116663, 1114050, 974766]","[6534178, 6533889, 1029743, 6534166, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[959987, 9803068, 919681, 5572234, 5567143]","[1075315, 925862, 878285, 979746, 886481]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [17]:
items_weights = weighted_top_5000[['item_id','sales_value']]
items_weights['weight'] = np.log(items_weights['sales_value'])
items_weights.loc[(items_weights['weight'] < 0)] = 0

min_value = items_weights['weight'].min()
max_value = items_weights['weight'].max()
sum_1 = items_weights['weight'].sum()
items_weights = items_weights[['item_id','weight']].groupby('item_id').sum()

items_weights['weight'] = items_weights['weight']/sum_1
items_weights['item_id'] = items_weights.index

items_weights.head(5)

,weight,item_id
item_id,,
0,0.000000,0
202291,0.000147,202291
397896,0.000266,397896
420647,0.000205,420647
480014,0.000226,480014


In [18]:
%%time
result['weighted_random_recommendation_top_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(5)

Wall time: 499 ms


,user_id,actual,weighted_random_recommendation,random_recommendation_top_5000,popular_recommendation_top_5000,weighted_random_recommendation_top_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12810466, 1147273, 824796, 7168833, 7025203]","[1080472, 933067, 9445919, 866950, 1007796]","[6534178, 6533889, 1029743, 6534166, 1082185]","[824555, 12731432, 879734, 875694, 5568845]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[823862, 7157113, 1075074, 941741, 2424234]","[9835509, 12352204, 947200, 965530, 986156]","[6534178, 6533889, 1029743, 6534166, 1082185]","[983316, 1077490, 1082770, 1119112, 1079048]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6533765, 1018818, 7440992, 1097661, 847962]","[867667, 863885, 1012043, 829617, 1072330]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1129610, 922847, 9837807, 898342, 1115335]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9487306, 1048445, 7406474, 985889, 847982]","[12349795, 6464056, 1116663, 1114050, 974766]","[6534178, 6533889, 1029743, 6534166, 1082185]","[833598, 12262955, 847207, 1094262, 1092303]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[959987, 9803068, 919681, 5572234, 5567143]","[1075315, 925862, 878285, 979746, 886481]","[6534178, 6533889, 1029743, 6534166, 1082185]","[963238, 1051069, 1032549, 989249, 844991]"
